In [ ]:
# xmlrpc_server.ipynb

import datetime
import pickle
import xmlrpc.client
import xmlrpc.server

import pandas as pd


class RequestHandler(xmlrpc.server.SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)


server = xmlrpc.server.SimpleXMLRPCServer(("localhost", 8008),
                                          requestHandler=RequestHandler)


# Тест
def ping():
    return True


server.register_function(ping, 'ping')


# Время сервера
def now():
    return datetime.datetime.now()


server.register_function(now, 'now')


# Отображение строкового вида, типа и значений
def show_type(arg):
    return (str(arg), str(type(arg)), arg)


server.register_function(show_type, 'type')


# Сумма
def test_sum(a, b):
    return a + b


server.register_function(test_sum, 'sum')


# Степень
def test_pow(a, b):
    return a ** b


server.register_function(test_pow, 'pow')


# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname):
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] == sname)
    if exist:
        return sname + ": " + "bad_boy"
    else:
        return sname + ": " + "good_boy"


server.register_function(black_list_check, 'black_list_check')


def black_list_check_by_fullname_and_birth(surname, patronym, name, birth):
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    if any((frame['Surname'] == surname) & (frame['Patronym'] == patronym) & (frame['Name'] == name) & (frame["Birth"] == birth)):
        return surname + " " + patronym + " " + name + " born " + birth + " is bad guy"
    else:
        return surname + " " + patronym + " " + name + " born " + birth + " not found"


server.register_function(black_list_check_by_fullname_and_birth, 'black_list_check_by_fullname')


# Бинарная передача данных
def send_back_binary(bin_data):
    data = bin_data.data
    return xmlrpc.client.Binary(data)


server.register_function(send_back_binary, 'send_back_binary')


def send_back_inversion(bin_data):

    img_arr = pickle.loads(bin_data.data)

    print(len(img_arr))
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    for i in range(height):
        for j in range(width):
            img_arr[i][j][0] = 255 - img_arr[i][j][0]
            img_arr[i][j][1] = 255 - img_arr[i][j][1]
            img_arr[i][j][2] = 255 - img_arr[i][j][2]

    pimg = pickle.dumps(img_arr)
    return xmlrpc.client.Binary(pimg)


server.register_function(send_back_inversion, 'color_inversion')


def image_binarization_by_limit(image_bin_data, limit):
    pass


server.register_function(image_binarization_by_limit, 'image_binarization_by_limit')

print("Listening on port 8008...")
server.serve_forever()